In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3

In [29]:
# Load and reshape the data
conn = sqlite3.connect('energy.db')

df = pd.read_sql_query('SELECT * FROM "Critical minerals (IEA) total demand"', conn)
df = df.melt(id_vars=['Mineral', 'Scenario'], var_name='Year', value_name='Total Demand')

conn.close()

In [30]:
# Select features and target variable
X = df[['Year', 'Mineral', 'Scenario']]
y = df['Total Demand']

In [31]:
# Convert categorical variables into dummy
X = pd.get_dummies(X)

In [32]:
# Split tand train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [33]:
# Make predictions and evaluate
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rounded_mse = round(mse, 2)
print("Mean Squared Error:", rounded_mse)

rmse = mse ** 0.5
rounded_rmse = round(rmse, 2)
print(f'The model is mistaken by {rounded_rmse} kt.')

Mean Squared Error: 1583510.03
The model is mistaken by 1258.38 kt.


In [34]:
minerals = ['Copper', 'Cobalt', 'Lithium', 'Nickle', 'Neodymium']
scenarios = ['Stated policies', 'Announced pledges', 'Zero Emissions']

fig = make_subplots(rows=len(minerals), cols=len(scenarios), 
                    subplot_titles=[f"{mineral} - {scenario}" for mineral in minerals for scenario in scenarios])

for i, mineral in enumerate(minerals):
    for j, scenario in enumerate(scenarios):
        # Prepare the input data for the current mineral and scenario
        input_data = pd.DataFrame({'Mineral': [mineral], 'Scenario': [scenario]})
        input_data = pd.concat([input_data]*len(df['Year'].unique()), ignore_index=True)
        input_data['Year'] = df['Year'].unique()
        input_data = pd.get_dummies(input_data)
        
        # Ensure all necessary dummy variables are present
        all_columns = set(X.columns)
        input_columns = set(input_data.columns)
        missing_columns = all_columns - input_columns
        for col in missing_columns:
            input_data[col] = 0

        # Reorder the columns to match the order during training
        input_data = input_data[X.columns]

        # Make predictions
        predictions = model.predict(input_data)

        # Get the real total demands
        real_demands = df[(df['Mineral'] == mineral) & (df['Scenario'] == scenario)]['Total Demand'].values

        # Add bar traces to the subplot
        fig.add_trace(go.Bar(x=df['Year'].unique(), y=predictions, name='Predicted'), row=i+1, col=j+1)
        fig.add_trace(go.Bar(x=df['Year'].unique(), y=real_demands, name='Real'), row=i+1, col=j+1)

fig.update_layout(title="Predicted vs Real Total Demand for Different Minerals and Scenarios",
                  xaxis_title="Year",
                  yaxis_title="Total Demand (kt)",
                  showlegend=False,
                  height=1200, width=1300)

fig.show()

In [35]:
import plotly.graph_objects as go

# Iterate over each mineral and scenario
for mineral in minerals:
    for scenario in scenarios:
        # Prepare the input data for the current mineral and scenario
        input_data = pd.DataFrame({'Mineral': [mineral], 'Scenario': [scenario]})
        input_data = pd.concat([input_data]*len(df['Year'].unique()), ignore_index=True)
        input_data['Year'] = df['Year'].unique()
        input_data = pd.get_dummies(input_data)

        all_columns = set(X.columns)
        input_columns = set(input_data.columns)
        missing_columns = all_columns - input_columns
        for col in missing_columns:
            input_data[col] = 0

        # Reorder the columns to match the order during training
        input_data = input_data[X.columns]

        # Make predictions
        predictions = model.predict(input_data)

        # Get the real total demands
        real_demands = df[(df['Mineral'] == mineral) & (df['Scenario'] == scenario)]['Total Demand'].values

        fig = go.Figure()
        
        # Add predicted data
        fig.add_trace(go.Scatter(x=df['Year'].unique(), y=predictions, mode='lines+markers', name='Predicted'))
        
        # Add real data
        fig.add_trace(go.Scatter(x=df['Year'].unique(), y=real_demands, mode='lines+markers', name='Real'))

        fig.update_layout(
            title='Predicted vs Real Total Demand for {} in {} Scenario'.format(mineral, scenario),
            xaxis_title='Year',
            yaxis_title='Total Demand',
            legend=dict(x=0, y=1),
            margin=dict(l=0, r=0, t=50, b=0),
            hovermode='x'
        )

        fig.show()

In [36]:
# Predict for a specific year, mineral, and scenario

selected_year = '2050'
selected_mineral = 'Cobalt'
selected_scenario = 'Announced pledges'

# Prepare the input data
input_data = pd.DataFrame({'Year': [selected_year], 'Mineral': [selected_mineral], 'Scenario': [selected_scenario]})
input_data = pd.get_dummies(input_data)

# Ensure all necessary dummy variables are present
all_columns = set(X.columns)
input_columns = set(input_data.columns)
missing_columns = all_columns - input_columns
for col in missing_columns:
    input_data[col] = 0

# Reorder the columns to match the order during training
input_data = input_data[X.columns]

# Make prediction
prediction = model.predict(input_data)

# Print the predicted total demand
print("\nFor Year:", selected_year)
print("Mineral:", selected_mineral)
print("Scenario:", selected_scenario)
print("Predicted Total Demand:", prediction[0])
real_demand = df[(df['Year'] == selected_year) & 
                 (df['Mineral'] == selected_mineral) & 
                 (df['Scenario'] == selected_scenario)]['Total Demand'].values[0]

print("Real Total Demand:", real_demand)


For Year: 2050
Mineral: Cobalt
Scenario: Announced pledges
Predicted Total Demand: 441.8130299999999
Real Total Demand: 524.779
